# Debugging autoreload

In [ ]:
%load_ext autoreload
%autoreload 2

# Load packages

In [4]:
from pytorch_tabular.utils import load_covertype_dataset
from rich.pretty import pprint
from sklearn.model_selection import BaseCrossValidator, ParameterGrid, ParameterSampler
import torch
import pickle
import shutil
from scipy import stats
import shap
from sklearn.model_selection import RepeatedStratifiedKFold
from glob import glob
import ast
import matplotlib.pyplot as plt
import seaborn as sns
import copy
import itertools
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import mean_absolute_error
from pytorch_tabular import MODEL_SWEEP_PRESETS
import pandas as pd
from pytorch_tabular import model_sweep
from src.pt.model_sweep import model_sweep_custom
import warnings
from src.utils.configs import read_parse_config
from src.pt.hyper_opt import train_hyper_opt
from src.utils.hash import dict_hash
import pathlib
from tqdm import tqdm
import distinctipy
import matplotlib.patheffects as pe
import matplotlib.colors as mcolors
from statannotations.Annotator import Annotator
from scipy.stats import mannwhitneyu
from plottable import ColumnDefinition, Table
from plottable.plots import bar
from plottable.cmap import normed_cmap, centered_cmap
import optuna
from matplotlib.colors import LinearSegmentedColormap
import matplotlib.cm
import matplotlib as mpl
from statsmodels.stats.multitest import multipletests
import re
from itertools import chain
from pathlib import Path
import requests
from matplotlib.ticker import MaxNLocator
from regression_bias_corrector import LinearBiasCorrector
from scipy.stats import mannwhitneyu, variation, levene
from copy import deepcopy
import functools


def conjunction(conditions):
    return functools.reduce(np.logical_and, conditions)


def disjunction(conditions):
    return functools.reduce(np.logical_or, conditions)

# Load data

In [7]:
path = f"E:/YandexDisk/Work/bbd/mriya"

df = pd.read_excel(f"{path}/data_bioage_all.xlsx", index_col=0)
df_icd = pd.read_excel(f"{path}/ICD10.xlsx", index_col=0)

cvds = [
    "E78.9", # Дислипидемия
    "I11.9", # гипертония
    "E66.0", # Ожирение
    # "G43.9", # Мигрень
    # "K26.9", # Язвенная болезнь
    # "K25.9", # Язвенная болезнь
    # "J45.9", # бронхиальная астма
    # "M06.9", # Ревматоидный артрит
    # "K76.0", # Жировая дегенерация печени
    "I48.9", # Фибрилляция предсердий и трепетание предсердий
    # "N11", # Хронический тубулоинтерстициальный нефрит
    # "M10.9", # Подагра
    "I25.9", # Хроническая ишемическая болезнь сердца
    # "L40.9", # Псориаз
    # "N18.9", # Хроническая болезнь почек
    # "J44.9", # Хроническая обструктивная легочная болезнь
    # "E47.3", # Недостаточность питания
    # "N48.4", # Импотенция органического происхождения
    "I25.2", # Перенесенный в прошлом инфаркт миокарда
    "I50.9", # Сердечная недостаточность
    # "I69.4", # Последствия инсульта
]
df['CVD'] = np.where(disjunction([df[m] == 1 for m in cvds]), 1, 0)
df['Heatlhy'] = np.where(conjunction([df[m] == 0 for m in df_icd.index]), 1, 0)

df.to_excel(f"{path}/07_select_samples_controls_and_CVD_cases/df.xlsx")
